# The Most ETL API

Примеры использования ETL API для загрузки метаданных коммуникаций.

ETL API позволяет загружать метаданные коммуникаций (звонки, чаты) пачкой для дальнейшей обработки и анализа.


In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime, timezone

sys.path.append('../')
from most import MostClient, CommunicationRequest


## Инициализация клиента

Для работы с ETL API используется тот же `MostClient`, но можно указать отдельный базовый URL для ETL через параметр `etl_base_url` или переменную окружения `MOST_ETL_BASE_URL`.


In [ ]:
# Инициализация клиента
# ETL base URL можно указать явно или через переменную окружения MOST_ETL_BASE_URL
client = MostClient(
    # etl_base_url="https://etl.the-most.ai"  # опционально, по умолчанию используется этот URL
)


## Загрузка коммуникаций с объектами CommunicationRequest

Можно передавать список объектов `CommunicationRequest`:


In [ ]:
now = datetime.now(timezone.utc)

communications = [
    CommunicationRequest(
        source_entity_id="crm_call_12345",
        most_communication_id="6904f6915a5dd2d4ca25beba",
        start_dt=now.isoformat(),
        end_dt=(now.replace(second=now.second + 300)).isoformat(),  # +5 минут
        manager="Иван Иванов",
        talk_duration=300,
        communication_type="call",
        communication_direction="inbound",
        communication_result="answered",
        client_phone="+79991234567",
        wait_duration=30,
    )
]

result = client.upload_communications(communications)
print(f"Сохранено коммуникаций: {result.total_saved}")
print(f"Статусы: {result.status_per_communication}")


## Загрузка коммуникаций со словарями

Также можно передавать список словарей - они автоматически валидируются и преобразуются в `CommunicationRequest`:


In [ ]:
communications_dict = [
    {
        "source_entity_id": "crm_chat_67890",
        "most_communication_id": "6904f6915a5dd2d4ca25beba",
        "start_dt": datetime.now(timezone.utc).isoformat(),
        "manager": "Петр Петров",
        "talk_duration": 180,
        "communication_type": "chat",
        "communication_direction": "outbound",
        "status": "completed",
    }
]

result = client.upload_communications(communications_dict)
print(f"Сохранено: {result.total_saved}")


## Пакетная загрузка нескольких коммуникаций

Можно загружать несколько коммуникаций одновременно:


In [ ]:
batch_communications = [
    CommunicationRequest(
        source_entity_id=f"call_{i}",
        most_communication_id=f"6904f6915a5dd2d4ca25beba-{i}",
        start_dt=datetime.now(timezone.utc).isoformat(),
        manager=f"Менеджер {i}",
        talk_duration=100 + i * 10,
        communication_type="call",
    )
    for i in range(5)
]

result = client.upload_communications(batch_communications)
print(f"Всего сохранено: {result.total_saved}")
print(f"Обработано коммуникаций: {len(result.status_per_communication)}")


## Загрузка с опциональными полями

Можно указать дополнительные поля для более детальной информации:


In [ ]:
detailed_communication = CommunicationRequest(
    source_entity_id="detailed_call_001",
    most_communication_id="6904f6915a5dd2d4ca25beba",
    start_dt=datetime.now(timezone.utc).isoformat(),
    end_dt=(datetime.now.replace(second=now.second + 450)).isoformat(),
    manager="Анна Смирнова",
    talk_duration=450,
    communication_type="call",
    communication_direction="inbound",
    communication_result="answered",
    client_phone="+79997654321",
    wait_duration=15,
    status="completed",
    manager_extension="1234",
    manager_direct_number="+79991111111",
    # Дополнительные поля, специфичные для клиента
    extra_fields={
        "campaign_id": "summer_2024",
        "product_category": "premium",
        "customer_segment": "vip"
    },
    # Технические поля
    tech_fields={
        "recording_url": "https://storage.example.com/recording.mp3",
        "call_quality": "high"
    }
)

result = client.upload_communications([detailed_communication])
print(f"Статус: {result.status_per_communication['detailed_call_001'].reason}")


## Перезапись существующих записей

С параметром `overwrite=True` можно перезаписать существующие записи:


In [ ]:
# Первая загрузка
communications = [
    CommunicationRequest(
        source_entity_id="update_test",
        most_communication_id="6904f6915a5dd2d4ca25beba",
        start_dt=datetime.now(timezone.utc).isoformat(),
        manager="Тестовый Менеджер",
        talk_duration=200,
        communication_type="call",
    )
]

result1 = client.upload_communications(communications, overwrite=False)
print(f"Первая загрузка: {result1.total_saved}")

# Обновление с overwrite=True
result2 = client.upload_communications(communications, overwrite=True)
print(f"Обновление: {result2.total_saved}")


## Обработка ошибок

При ошибках API выбрасывается `RuntimeError` с понятным сообщением:


In [ ]:
try:
    # Пример с неполными данными (вызовет ошибку валидации)
    invalid_communications = [
        {
            "source_entity_id": "invalid",
            # Отсутствуют обязательные поля
        }
    ]
    client.upload_communications(invalid_communications)
except RuntimeError as e:
    print(f"Ошибка: {e}")
except Exception as e:
    print(f"Ошибка валидации: {e}")


## Примечания

- Для работы с ETL API клиент должен быть зарегистрирован в системе ETL
- Если клиент не зарегистрирован, будет выброшена ошибка с соответствующим сообщением
- Все даты должны быть в ISO формате (например, `datetime.now(timezone.utc).isoformat()`)